In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from astropy.table import Table
import pyvo as vo

In [ ]:
import groupten

In [ ]:
simbadtap = vo.dal.TAPService("http://simbad.u-strasbg.fr/simbad/sim-tap")

In [ ]:
df = groupten.load_targets()

In [ ]:
def fixhipstring(hipstring):
    """pad spaces between to fill 10 character
    This is hack to make simbad join happen."""
    ids = hipstring.split()[-1]
    return "HIP{:>7s}".format(ids)

In [ ]:
uploadtable = Table({'name': [fixhipstring(s) if 'HIP' in s else s for s in df.hip_tyc_id.values]})
uploadtable.write('tmp.xml', format='votable', overwrite=True)

In [ ]:
result = simbadtap.run_sync("""
select t1.name, id2.id
from tap_upload.t1 as t1 left join ident as id1 on id1.id = t1.name
join ident as id2 using (oidref)
where id2.id != id1.id
""", uploads={"t1":('inline', 'tmp.xml')})

In [ ]:
dfiden = result.table.to_pandas()

In [ ]:
dfiden.apply(lambda x: x.str.decode('utf-8'))